In [1]:
import os
import numpy as np 
import pandas as pd

In [2]:
sub_path = "/home/gabriel/Desktop/BIT_AI_ICEBERG/input"
all_files = os.listdir(sub_path)

### Load the submissions and concatenate

In [3]:
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5
0,5941774d,0.021835,0.005586,1.554056e-02,1.943216e-02,0.023567,0.003100
1,4023181e,0.344095,0.145927,5.970687e-01,3.168809e-02,0.958522,0.791638
2,b20200e4,0.004762,0.000015,2.644745e-18,4.000000e-08,0.009161,0.033046
3,e7f018bb,0.997243,0.999914,9.782760e-01,9.925741e-01,0.974719,0.988451
4,4371c8c3,0.138532,0.033843,4.948649e-02,2.215107e-02,0.494530,0.410258


### Correlation

In [4]:
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5
is_iceberg_0,1.000000,0.539599,0.719763,0.436504,0.582096,0.838629
is_iceberg_1,0.539599,1.000000,0.574108,0.890434,0.644485,0.543359
is_iceberg_2,0.719763,0.574108,1.000000,0.498916,0.679996,0.741193
is_iceberg_3,0.436504,0.890434,0.498916,1.000000,0.595725,0.452017
is_iceberg_4,0.582096,0.644485,0.679996,0.595725,1.000000,0.650091
is_iceberg_5,0.838629,0.543359,0.741193,0.452017,0.650091,1.000000


### Stacking

In [5]:
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1) # 6 to liczba plikow, ale mozna wybrac dowolnie
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

### Cutoff values

In [6]:
cutoff_lo = 0.8
cutoff_hi = 0.2

### Mean

In [ ]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', 
                                        index=False, float_format='%.6f')

### Median

In [ ]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', 
                                        index=False, float_format='%.6f')

### Minmax + mean

In [ ]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

### Minmax + median

In [ ]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

### MinMax + BestBase

In [7]:
# load the model with best base performance
sub_base = pd.read_csv("/home/gabriel/Desktop/BIT_AI_ICEBERG/base_kag.csv")

In [8]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')